In [19]:
ticker = input('Please input the ticker of the stock: ')

In [38]:
def analysis(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/analysis?p' + ticker
    response = requests.get(link)
    if response.status_code == 200:
        pass
    else:
        print('An error occurred when accessing the page!')
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    analysis_info = dict()
    for table_tag in result_page.find_all('table'):
        col,data_col,data_frame = [], [], []
        for th_tag in table_tag.find_all('th'):
            col.append(th_tag.get_text())
        i = 1
        for td_tag in table_tag.find_all('td'):
            if 1 <= i <= 5:
                data_col.append(td_tag.get_text())
                i += 1
            else:
                data_frame.append(data_col)
                data_col = []
                i = 1
                data_col.append(td_tag.get_text())
                i += 1
        data_frame.append(data_col)
        df = pd.DataFrame(data_frame,columns = col)
        df.set_index(col[0],inplace=True)
        df_list.append(df)
        analysis_info[col[0]] = df
    return analysis_info

In [50]:
def holders(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/holders?p' + ticker
    response = requests.get(link)
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    for div_tag in result_page.find_all('div'):
        if div_tag.get('class') in [['Mt(25px)', 'Ovx(a)', 'W(100%)'],['W(100%)', 'Mb(20px)']]:
            if div_tag.find('span').get_text() == 'Breakdown':
                j = 2
                col = ['Major Holders Breakdown',' ']
            else:
                j = 5
                col = []
            i = 1
            data_col,data_frame = [],[]
            for th_tag in div_tag.find_all('th'):
                col.append(th_tag.get_text())
            for td_tag in div_tag.find_all('td'):
                if 1 <= i <= j:
                    data_col.append(td_tag.get_text())
                    i += 1
                else:
                    data_frame.append(data_col)
                    data_col = []
                    i = 1
                    data_col.append(td_tag.get_text())
                    i += 1
            data_frame.append(data_col)
            df = pd.DataFrame(data_frame,columns = col)
            df.set_index(col[0],inplace=True)
            df_list.append(df)
    df_list[1].index.names = ['Top Institutional Holders']
    df_list[2].index.names = ['Top Mutual Fund Holders']
    holders_info = {'major':df_list[0],'institutional':df_list[1],'mutual fund':df_list[2]}
    return holders_info

In [22]:
def get_financial(ticker,IS=0,BS=0,CF=0):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    IS_keyword = ['Revenue','Operating Expenses','Income from Continuing Operations','Non-recurring Events','Net Income']
    BS_keyword = ['Assets','Liabilities','Equity']
    CF_keyword = ['Net Income','Operating Activities','Investing Activities','Financing Activities','Exchange Rate Changes']
    link = 'https://finance.yahoo.com/quote/' + ticker + '/financials?p=' + ticker
    keyword = IS_keyword
    if BS:
        link = 'https://finance.yahoo.com/quote/' + ticker + '/balance-sheet?p=' + ticker
        keyword = BS_keyword
    if CF:
        link = 'https://finance.yahoo.com/quote/' + ticker + '/cash-flow?p=' + ticker
        keyword = CF_keyword
    response = requests.get(link)
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    col,data_col,data_frame = [None], [], []
    i = 1
    end = 0
    for tr_tag in result_page.find_all('tr'):
        for td_tag in tr_tag.find_all('td'):
            if td_tag.get('class') == ['C($gray)', 'Ta(end)']:
                col.append(td_tag.get_text())
            if not td_tag.get('class') in [['Fw(b)', 'Fz(15px)'],['C($gray)', 'Ta(end)'],['Fw(b)', 'Fz(15px)', 'Pb(8px)', 'Pt(36px)']]:
                if 1 <= i <= 5:
                    data_col.append(td_tag.get_text())
                    i += 1
                else:
                    data_frame.append(data_col)
                    data_col = []
                    i = 1
                    data_col.append(td_tag.get_text())
                    i += 1
            if tr_tag.get('class') == ['Bdbw(0px)!', 'H(36px)']:
                end += 1
            if end == 5:
                data_frame.append(data_col)
                df = pd.DataFrame(data_frame)
                df_list.append(df)
                end = 0
                i = 1
                data_col,data_frame = [], []
    for k in range(len(keyword)):
        col_name = col
        col_name[0] = keyword[k] + ' (All numbers in thousands)'
        df_list[k].columns = col_name
        df_list[k].set_index(col_name[0],inplace=True)
    return df_list

In [52]:
holders(ticker)

{'major':                                                               
 Major Holders Breakdown                                       
 24.58%                         % of Shares Held by All Insider
 62.77%                        % of Shares Held by Institutions
 83.23%                         % of Float Held by Institutions
 1,409                    Number of Institutions Holding Shares,
 'institutional':                                            Shares Date Reported   % Out  \
 Top Institutional Holders                                                 
 Mitsubishi UFJ Financial Group, Inc.  425,706,748  Jun 29, 2018  24.40%   
 State Street Corporation              132,386,534  Jun 29, 2018   7.59%   
 Blackrock Inc.                        109,110,758  Jun 29, 2018   6.25%   
 Vanguard Group, Inc. (The)             95,216,528  Jun 29, 2018   5.46%   
 Price (T.Rowe) Associates Inc          87,399,650  Jun 29, 2018   5.01%   
 JP Morgan Chase & Company              47,257,747  Jun 2